### Подключим все необходимые библиотеки

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from collections import Counter
import re
import nltk
#import pymorphy2
from ast import literal_eval
import gensim


### Поля данных

* warehouse_id - идентификатор магазина
* product_id - идентификатор продукта
* date - дата
* quantity - кол-во продаж
* id - уникальный идентификатор строки

 

In [24]:
train_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/train.csv")
print(train_data.shape)
train_data.head()

(78067, 5)


,warehouse_id,product_id,date,quantity,id
0,0,71165,2020-12-02,1,0
1,0,71165,2020-12-03,3,1
2,0,71165,2020-12-07,2,2
3,0,71165,2020-12-08,2,3
4,0,71165,2020-12-09,3,4


### Посмотрим на количество товаров и на их id

In [27]:
product_id = train_data.sort_values(by=['product_id'])['product_id'].unique()
print(len(product_id))
print(product_id[:5])

3288
[71165 71170 71185 71215 71220]


### Посмотрим на тестовую выборку и сделаем некоторые выводы

In [26]:
test_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/test.csv")
print(test_data.shape)
test_data.head(15)

(46032, 4)


,date,product_id,warehouse_id,id
0,2021-04-09,71165,0,0
1,2021-04-09,71165,1,1
2,2021-04-09,71170,0,2
3,2021-04-09,71170,1,3
4,2021-04-09,71185,0,4
5,2021-04-09,71185,1,5
6,2021-04-09,71215,0,6
7,2021-04-09,71215,1,7
8,2021-04-09,71220,0,8
9,2021-04-09,71220,1,9


### Сразу хочется посмотреть на warehouse_id и на product_id, так как при разных product_id будут проблемы


In [34]:
product_test_id = test_data.sort_values(by=['product_id'])['product_id'].unique()
print(len(product_test_id))
print(product_test_id[:5])
print(len(set(product_test_id == product_id)))  # Все значения совпадают, проблем нет

3288
[71165 71170 71185 71215 71220]
1


In [37]:
warehouse_id = test_data['warehouse_id'].unique()
print(len(warehouse_id))  # Всего два магазина

2


### В нашем случае всего два магазина и все значения id товаров совпадают. Проблем с данными нет.

### Посмотрим на simple submission

In [23]:
simple_data = pd.read_csv("/content/drive/MyDrive/Sales_Forecasting/sub.csv")
print(simple_data.shape)
simple_data.head()


(46032, 2)


,id,quantity
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


3288
[71165 71170 71185 71215 71220]
